# **Embedding Nedir?**

# Bu çalışmada, milyonlarca yüzle önceden eğitilmiş güçlü bir derin öğrenme modeli olan ArcFace'i kullanarak yeni bir model eğitmek yerine, yüzünüzü sisteme "tanıştırma" (özellik çıkarımı) işlemini gerçekleştiriyoruz. Koleksiyonunuzdaki her bir fotoğrafı modelimizden geçirerek yüzünüzün matematiksel özeti olan 512 boyutlu dijital parmak izlerini (embedding) elde ediyoruz; ardından bu farklı örneklerin ortalamasını alarak ışık veya açı gibi dış etkenlerden arındırılmış, en kararlı referans vektörümüzü oluşturuyoruz. Son aşamada yaptığımız normalizasyon işlemiyle de bu dijital kimliğinizi, C++ tarafında kameradan gelen anlık görüntülerle saniyeler içinde ve yüksek doğrulukla kıyaslanabilecek profesyonel bir veri yapısına dönüştürüyoruz.


#Embedding, bir yüzün matematiksel parmak izidir. Model (ArcFace), karmaşık bir yüz fotoğrafını alıp onu 512 tane sayıdan oluşan bir vektöre dönüştürür. Bu sayılar, yüzündeki karakteristik özellikleri (göz mesafesi, burun yapısı vb.) dijital bir imza haline getirir.

# **Neden Sınıflandırma (YOLOv8) Değil de "Embedding" Kullanıyoruz?**

# YOLOv8 gibi standart sınıflandırma modelleri nesneyi "etiketlemek" üzerine kuruludur; bu yüzden sisteme yeni bir kişi eklemek istediğinizde tüm modeli binlerce fotoğraflarla baştan eğitmeniz gerekir. Biz ise ArcFace ile yüzü 512 boyutlu evrensel bir sayısal parmak izine (embedding) dönüştürüyoruz.

# Bu yöntem sayesinde modelimizi hiçbir zaman yeniden eğitmeden, sadece matematiksel imzaları kıyaslayarak yeni kişileri saniyeler içinde sisteme tanıtabiliyoruz. Sonuç olarak; hantal eğitim süreçlerinden kurtulmuş, milyonlarca kişiye aynı hızla ölçeklenebilen, esnek ve profesyonel bir dijital kimlik altyapısı inşa ediyoruz.

In [ ]:
!pip install insightface onnxruntime torch torchvision -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 31.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.5 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from numpy.linalg import norm
import os

# Bu satırda, sistemin hangi modelleri kullanacağını ve bu modellerin nerede çalışacağını tanımlıyoruz.


In [ ]:
app = FaceAnalysis(
    name="buffalo_l",     # ArcFace + detector + landmark
    providers=['CPUExecutionProvider']
)
# buffalo_l doğrudan ArcFace mimarisini kullanan ve InsightFace kütüphanesi tarafından sunulan en güçlü model paketlerinden biri
app.prepare(ctx_id=0, det_size=(640, 640))

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 89358.89KB/s]


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


# Ortalama Almanın Mantığı: Klasörümüzdeki tüm fotoğraflardan ayrı ayrı embedding'ler çıkarıp bunların ortalamasını (np.mean) alıyoruz. Bu sayede farklı ışık, açı veya mimiklerden kaynaklanan hataları temizleyerek, yüzümüzü temsil eden en kararlı ve kusursuz "referans vektörü" oluşturuyoruz.

#Normalizasyon: En sonda yaptığımız norm işlemi, bu 512 sayıyı "birim boyuta" getiriyor. Bu, C++ tarafında anlık gelen yüzle bu referans yüzü karşılaştırırken (Cosine Similarity) hesaplamanın çok daha hızlı ve doğru yapılmasını sağlar.

In [ ]:
REF_DIR = "/content/drive/MyDrive/dataset_face/me"
embeddings = []

for img_name in os.listdir(REF_DIR):
    img = cv2.imread(os.path.join(REF_DIR, img_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    faces = app.get(img)
    if len(faces) == 0:
        continue

    embeddings.append(faces[0].embedding)

ref_embedding = np.mean(embeddings, axis=0)
ref_embedding = ref_embedding / norm(ref_embedding)

print("Referans embedding hazır:", ref_embedding.shape)

Referans embedding hazır: (512,)


# Tanıma aşamasında, kameradan gelen anlık yüz vektörünüz ile sistemimize daha önce kaydettiğimiz referans vektörünüz arasındaki benzerliği Cosine Similarity (Kosinüs Benzerliği) yöntemiyle ölçüyoruz. Burada odak noktamız noktalar arasındaki düz mesafe değil, bu iki vektör arasındaki "açısal yakınlıktır." Bu yaklaşım sayesinde, ortamdaki ışık şiddeti veya gölge durumu değişse bile yüzünüzün karakteristik yapısı (yani vektörün yönü) değişmediği için sizi her türlü koşulda yüksek doğrulukla ve hızla tanıyabiliyoruz.

In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

In [ ]:
def verify_face(image_path, threshold=0.6):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    faces = app.get(img)
    if len(faces) == 0:
        return False, 0.0

    emb = faces[0].embedding
    emb = emb / norm(emb)

    score = cosine_similarity(ref_embedding, emb)
    return score > threshold, score

In [ ]:
result, score = verify_face("/content/drive/MyDrive/test.jpg")

print("Eşleşti mi:", result)
print("Benzerlik:", score)

Eşleşti mi: True
Benzerlik: 0.6808997


In [ ]:
import numpy as np
np.save("my_face.npy", ref_embedding)
print("Python için embedding kaydedildi: my_face.npy")

# Bu veriyi C++ projemize doğrudan gömebilmek (hardcoded) için bir header (my_face.h) dosyasına dönüştürdük; bu sayede diskten dosya okuma gecikmesini (I/O lag) tamamen ortadan kaldırdık.

In [ ]:
import numpy as np

# Embedding'i listeye çeviriyoruz
embedding_list = ref_embedding.flatten().tolist()

# C++ Header dosyası oluşturma
header_content = f"""#ifndef MY_FACE_H
#define MY_FACE_H

// Auto-generated embedding vector
const float my_face_embedding[512] = {{
    {', '.join(map(str, embedding_list))}
}};

#endif
"""

with open("my_face.h", "w") as f:
    f.write(header_content)

print("C++ Header dosyası oluşturuldu: my_face.h")

C++ Header dosyası oluşturuldu: my_face.h
